In [222]:
import pandas as pd
import numpy as np
import networkx as nx
from datetime import datetime, timedelta

from entsoe import EntsoePandasClient
from entsoe.mappings import Area, NEIGHBOURS, lookup_area
from entsoe.parsers import parse_generation
from typing import Union, Optional, Dict, List, Literal
import zipfile
import bs4
from io import BytesIO

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [153]:
k_closest = 10

start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
end = pd.Timestamp('20230331', tz='Europe/Amsterdam')

countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

fbmc_borders = [
    ["NL", "DE"],
    ["NL", "BE"],
    ["BE", "DE"],
    ["BE", "NL"],
    ["BE", "FR"],
    ["FR", "DE"],
    ["DE", "PL"],
    ["DE", "CZ"],
    ["DE", "AT"],
    ["PL", "CZ"],
    ["PL", "SK"],
    ["CZ", "AT"],
    ["CZ", "SK"],
    ["HU", "AT"],
    ["HU", "RO"],
    ["HU", "HR"],
    ["HU", "SI"],
    ["AT", "SI"],
    ["SI", "HR"]
]

In [138]:
df_plants = pd.read_excel('./plants.xlsx', index_col=0)

In [3]:
df_ptdf = pd.read_excel('./ptdf_z_obs.xlsx', index_col=0)

In [7]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)
df_grid = df_grid[df_grid.susceptance != 0] # leave out zero length network elements
df_grid['susceptance'] = df_grid['susceptance'].astype(float)

df_substations = pd.read_excel('./substations.xlsx', index_col=0)

Grid = nx.from_pandas_edgelist(df_grid, 'substation_1', 'substation_2', edge_attr=["susceptance"], create_using=nx.MultiGraph)

In [4]:
display(df_grid)
display(df_ptdf)

,name,eic,tso,substation_1,substation_2,voltage,resistance,reactance,susceptance,length,zone,tieline,max_current
line_id,,,,,,,,,,,,,
0,Eula - Röhrsdorf 203,11TD8L203------C,50HERTZ,744,570,220,4.0937,18.349,161.163703,51.511,DE,False,790.00
1,Röhrsdorf - Weida 207,11TD8L207------H,50HERTZ,570,148,220,7.5895,31.888,348.716785,98.324,DE,False,790.00
2,Eula - Weida 208,11TD8L208------9,50HERTZ,744,148,220,5.0537,19.541,264.239358,67.550,DE,False,1070.00
3,Bentwisch - Güstrow 275,11TD8L275------7,50HERTZ,695,167,220,1.9090,12.055,158.084942,41.418,DE,False,1200.00
4,Wuhlheide - Thyrow 291,11TD8L291------L,50HERTZ,687,6,220,2.9672,11.191,266.784000,38.882,DE,False,960.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,Buers - Westtirol rt (422),10T-AT-DE-000053,TRANSNETBW,870,794,400,0.9900,14.270,168.703500,91.400,DE,True,2300.00
2848,Buers - Westtirol ws (421),10T-AT-DE-000061,TRANSNETBW,870,794,220,1.0000,14.365,170.117300,92.000,DE,True,2140.00
2849,Buers - Y-Werben,11T-D4-D7-00001W,TRANSNETBW,870,987,220,0.5950,90.800,70.418810,47.000,DE,True,1710.00


,DateTime,line_id,contingency,fref,frm,ram,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
0,2022-09-01 00:00:00,12,-1,177.0,42.0,153.0,-0.04836,-0.04861,-0.01764,-0.04791,-0.01287,-0.04590,-0.04244,-0.00756,0.00085,-0.05174,0.04220,0.00071,-0.01033,0.00715
1,2022-09-01 00:00:00,94,9,863.0,173.0,653.0,-0.01960,-0.02016,-0.02146,-0.01994,-0.07237,-0.03774,-0.02340,-0.00799,-0.00063,-0.01585,0.08202,0.00082,-0.01122,0.00769
2,2022-09-01 00:00:00,152,8,491.0,206.0,1140.0,-0.12361,-0.09008,0.00777,-0.13408,0.00672,-0.00222,-0.14070,-0.00395,0.00176,-0.07122,0.00364,0.00030,-0.00444,0.00304
3,2022-09-01 00:00:00,223,7,1243.0,207.0,919.0,-0.08881,-0.11644,0.01279,-0.08286,-0.02146,-0.00737,0.00638,0.00944,-0.00309,-0.09761,-0.02508,-0.00078,0.01233,-0.00938
4,2022-09-01 00:00:00,333,6,692.0,149.0,887.0,0.08342,0.08146,-0.01833,0.08480,-0.00646,0.05803,0.09712,0.00762,-0.00440,0.07500,0.00517,-0.00053,0.00870,-0.00457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299551,2023-03-31 22:00:00,2833,267,130.0,140.0,983.0,0.08168,0.08136,0.05098,0.08213,-0.07219,0.07571,0.08407,0.01285,-0.00404,0.07844,-0.00940,-0.00124,0.02356,-0.01421
299552,2023-03-31 22:00:00,2842,268,-68.0,45.0,262.0,0.04749,0.04863,-0.01230,0.04720,0.01530,0.04659,0.04495,-0.00054,-0.00184,0.04800,0.01892,0.00011,-0.00013,0.00249
299553,2023-03-31 22:00:00,2844,269,-68.0,45.0,262.0,0.04748,0.04862,-0.01230,0.04720,0.01530,0.04658,0.04495,-0.00054,-0.00184,0.04800,0.01892,0.00011,-0.00013,0.00249
299554,2023-03-31 22:00:00,2850,-1,764.0,251.0,908.0,-0.07457,-0.03530,0.00516,-0.08367,0.00179,0.00098,-0.09946,-0.00087,0.00050,-0.04448,-0.00075,0.00008,-0.00168,0.00058


In [121]:
#df_grid["closest_nodes_" + str(k_closest)] = df_grid["closest_nodes_" + str(k_closest)].astype('object')
#df_grid["closest_edges_" + str(k_closest)] = df_grid["closest_edges_" + str(k_closest)].astype('object')

k_closest_nodes_list = []
k_closest_edges_list = []

for line_id, line in df_grid.iterrows():
    source_node = line['substation_1']
 
    k_closest_nodes = list(nx.single_source_shortest_path_length(Grid ,source=source_node, cutoff=k_closest).keys())[0:k_closest]

    k_closest_edges = []
    for k in range(1, k_closest):
        nodes = list(nx.single_source_shortest_path_length(Grid ,source=source_node, cutoff=k_closest).keys())[0:k]
        edges = Grid.subgraph(nodes).edges()
    
        for edge in edges:
            line_ids = list(df_grid[((df_grid.substation_1 == edge[0]) & (df_grid.substation_2 == edge[1])) | ((df_grid.substation_2 == edge[0]) & (df_grid.substation_1 == edge[1]))].index)
            for line_id in line_ids:
                if line_id not in k_closest_edges and len(k_closest_edges) < k_closest - 1:
                    k_closest_edges.append(line_id)

    k_closest_nodes_list.append(k_closest_nodes)
    k_closest_edges_list.append(k_closest_edges)


In [126]:
df_grid["closest_nodes_" + str(k_closest)] = k_closest_nodes_list
df_grid["closest_edges_" + str(k_closest)] = k_closest_edges_list

In [128]:
display(df_grid)
df_grid.to_excel('./grid_with_k_closest.xlsx')

,name,eic,tso,substation_1,substation_2,voltage,resistance,reactance,susceptance,length,zone,tieline,max_current,closest_nodes_10,closest_edges_10
line_id,,,,,,,,,,,,,,,
0,Eula - Röhrsdorf 203,11TD8L203------C,50HERTZ,744,570,220,4.0937,18.349,161.163703,51.511,DE,False,790.00,"[744, 570, 148, 172, 286, 950, 598, 1143, 1223...","[0, 2, 1, 106, 21, 22, 109, 110, 104]"
1,Röhrsdorf - Weida 207,11TD8L207------H,50HERTZ,570,148,220,7.5895,31.888,348.716785,98.324,DE,False,790.00,"[570, 744, 148, 286, 598, 1143, 1223, 778, 172...","[0, 2, 1, 106, 109, 110, 104, 105, 108]"
2,Eula - Weida 208,11TD8L208------9,50HERTZ,744,148,220,5.0537,19.541,264.239358,67.550,DE,False,1070.00,"[744, 570, 148, 172, 286, 950, 598, 1143, 1223...","[0, 2, 1, 106, 21, 22, 109, 110, 104]"
3,Bentwisch - Güstrow 275,11TD8L275------7,50HERTZ,695,167,220,1.9090,12.055,158.084942,41.418,DE,False,1200.00,"[695, 167, 898, 422, 1396, 1295, 436, 554, 793...","[3, 80, 81, 17, 18, 15, 16, 19, 29]"
4,Wuhlheide - Thyrow 291,11TD8L291------L,50HERTZ,687,6,220,2.9672,11.191,266.784000,38.882,DE,False,960.00,"[687, 6, 141, 720, 592, 262, 207, 913, 761, 1342]","[4, 6, 7, 8, 69, 70, 68, 50, 51]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,Buers - Westtirol rt (422),10T-AT-DE-000053,TRANSNETBW,870,794,400,0.9900,14.270,168.703500,91.400,DE,True,2300.00,"[870, 987, 1498, 1499, 794, 1506, 205, 1130, 9...","[154, 177, 2849, 2698, 2845, 2705, 2706, 2707,..."
2848,Buers - Westtirol ws (421),10T-AT-DE-000061,TRANSNETBW,870,794,220,1.0000,14.365,170.117300,92.000,DE,True,2140.00,"[870, 987, 1498, 1499, 794, 1506, 205, 1130, 9...","[154, 177, 2849, 2698, 2845, 2705, 2706, 2707,..."
2849,Buers - Y-Werben,11T-D4-D7-00001W,TRANSNETBW,870,987,220,0.5950,90.800,70.418810,47.000,DE,True,1710.00,"[870, 987, 1498, 1499, 794, 1506, 205, 1130, 9...","[154, 177, 2849, 2698, 2845, 2705, 2706, 2707,..."


In [139]:
df_production_outages = pd.DataFrame()

for country_code in [x for x in countries if x not in ['HR', 'SI']]:
    print(country_code)
    df_outages_temp = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
    df_outages_temp = df_outages_temp.tz_localize(None)
    df_outages_temp['end'] = df_outages_temp['end'].dt.tz_localize(None)
    df_outages_temp['start'] = df_outages_temp['start'].dt.tz_localize(None)
    df_production_outages = pd.concat([df_production_outages, df_outages_temp], ignore_index=True)

display(df_production_outages)

AT
BE
CZ
DE_LU
FR
HU
NL
PL
RO
SK


,avail_qty,biddingzone_domain,businesstype,curvetype,docstatus,end,mrid,nominal_power,plant_type,production_resource_id,production_resource_location,production_resource_name,pstn,qty_uom,resolution,revision,start
0,389,AT,Planned maintenance,A03,Cancelled,2022-09-28 00:00:00,AG_5D27uQcpgx6crlpQsKg,840.0,Fossil Gas,14W-PROD-SIM---P,intra_zonal,KW Simmering,1,MAW,PT15M,3,2022-09-16 00:00:00
1,0,AT,Planned maintenance,A03,Cancelled,2024-01-01 00:00:00,_LluwDqTiRSOQFu38pGrGw,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,2,2023-01-01 00:00:00
2,0,AT,Planned maintenance,A03,Cancelled,2023-01-01 00:00:00,be3Cxp6yr6iQNN07LZSgJA,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,2,2022-01-01 00:00:00
3,0,AT,Planned maintenance,A03,None,2024-01-01 00:00:00,RLThCIJuxMYLJYdEKwL7eQ,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,1,2023-01-01 00:00:00
4,0,AT,Planned maintenance,A03,None,2023-01-01 00:00:00,KqPp9Q65jWB93v_xwyKxEA,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,1,2022-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16422,0,SK,Planned maintenance,A03,None,2023-02-24 12:00:00,5XX-YkWTHcEd7oTClJTm4w,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,39,2023-02-24 11:00:00
16423,0,SK,Planned maintenance,A03,None,2023-02-24 10:00:00,6zbRH9yTF8UGN9SVt16lEA,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,39,2023-02-24 09:00:00
16424,0,SK,Planned maintenance,A03,None,2023-01-24 14:00:00,0pDus7KsJotf7w5SgjOL8A,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,68,2023-01-24 07:00:00
16425,0,SK,Planned maintenance,A03,None,2023-03-01 17:00:00,9v_NDHwUlLbKn7RYoiAqSQ,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,39,2023-03-01 08:00:00


In [140]:
df_production_outages.to_excel('./production_outages.xlsx')

In [249]:
def _unavailability_tm_ts(soup: bs4.BeautifulSoup) -> list:
    # Avoid attribute errors when some of the fields are void:
    get_attr = lambda attr: "" if soup.find(attr) is None else soup.find(
        attr).text
    # When no nominal power is given, give default numeric value of 0:

    f = [BSNTYPE[get_attr('businesstype')],
         _INV_BIDDING_ZONE_DICO[get_attr('in_domain.mrid')],
         _INV_BIDDING_ZONE_DICO[get_attr('out_domain.mrid')],
         get_attr('quantity_measure_unit.name'),
         soup.find('asset_registeredresource').mrid.text,
         ]
    return [f + p for p in _available_period(soup)]
    
def _outage_parser(xml_file: bytes, headers, ts_func) -> pd.DataFrame:
    xml_text = xml_file.decode()

    soup = bs4.BeautifulSoup(xml_text, 'html.parser')
    mrid = soup.find("mrid").text
    revision_number = int(soup.find("revisionnumber").text)
    try:
        creation_date = pd.Timestamp(soup.createddatetime.text)
    except AttributeError:
        creation_date = ""

    try:
        docstatus = DOCSTATUS[soup.docstatus.value.text]
    except AttributeError:
        docstatus = None
    d = list()
    series = _extract_timeseries(xml_text)
    for ts in series:
        row = [creation_date, docstatus, mrid, revision_number]
        for t in ts_func(ts):
            d.append(row + t)
    df = pd.DataFrame.from_records(d, columns=headers)
    return df
    
def query_unavailability_transmission(country_code_from, country_code_to, start, end):
    area_in = lookup_area(country_code_from)
    area_out = lookup_area(country_code_to)
    params = {
        'documentType': "A78",
        'in_Domain': area_in.code,
        'out_Domain': area_out.code,
        'offset': 0
    }
    response = client._base_request(params=params, start=start, end=end)
    _UNAVAIL_PARSE_CFG = {
          'A78': ([
          'created_doc_time',
          'docstatus',
          'mrid',
          'revision',
          'businesstype',
          'in_domain',
          'out_domain',
          'qty_uom',
          'eic',
          'start',
          'end',
          'resolution',
          'pstn',
          'avail_qty'
        ], _unavailability_tm_ts),
    }
    
    headers, ts_func = _UNAVAIL_PARSE_CFG["A78"]
    dfs = list()
    with zipfile.ZipFile(BytesIO(response.content), 'r') as arc:
        for f in arc.infolist():
            if f.filename.endswith('xml'):
                frame = _outage_parser(arc.read(f), headers, ts_func)
                dfs.append(frame)
    if len(dfs) == 0:
        df = pd.DataFrame(columns=headers)
    else:
        df = pd.concat(dfs, axis=0)
    df.set_index('created_doc_time', inplace=True)
    df.sort_index(inplace=True)
    
    return df

In [250]:
df_transmission_outages = pd.DataFrame()

for border in fbmc_borders:
    print(border)
    try:
        df_outages_temp = query_unavailability_transmission(border[0], border[1], start=start, end=end)
        df_outages_temp = df_outages_temp.tz_localize(None)
        df_outages_temp['end'] = df_outages_temp['end'].dt.tz_localize(None)
        df_outages_temp['start'] = df_outages_temp['start'].dt.tz_localize(None)
        df_transmission_outages = pd.concat([df_transmission_outages, df_outages_temp], ignore_index=True)
    except:
        print('Skipping')

display(df_transmission_outages)

['NL', 'DE']
Skipping
['NL', 'BE']
['BE', 'DE']
Skipping
['BE', 'NL']
['BE', 'FR']
['FR', 'DE']
Skipping
['DE', 'PL']
Skipping
['DE', 'CZ']
Skipping
['DE', 'AT']
Skipping
['PL', 'CZ']
['PL', 'SK']
['CZ', 'AT']
['CZ', 'SK']
['HU', 'AT']
['HU', 'RO']
Skipping
['HU', 'HR']
Skipping
['HU', 'SI']
Skipping
['AT', 'SI']
Skipping
['SI', 'HR']
Skipping


,docstatus,mrid,revision,businesstype,in_domain,out_domain,qty_uom,eic,start,end,resolution,pstn,avail_qty
0,Cancelled,SXpyAUdpXIjTMAJ2IkDk4A,2,Planned maintenance,NL,BE,MAW,22T-BE-IN-LI006Y,2022-06-27 05:30:00,2022-06-27 06:00:00,PT15M,1,950
1,Cancelled,SXpyAUdpXIjTMAJ2IkDk4A,2,Planned maintenance,NL,BE,MAW,22T-BE-IN-LI006Y,2022-06-27 06:00:00,2022-09-23 13:00:00,PT60M,1,950
2,Cancelled,SXpyAUdpXIjTMAJ2IkDk4A,2,Planned maintenance,NL,BE,MAW,22T-BE-IN-LI006Y,2022-09-23 13:00:00,2022-09-23 13:30:00,PT15M,1,950
3,Cancelled,TQkAi1gcCs2HwFhTeB9a_w,2,Planned maintenance,NL,BE,MAW,22T-BE-IN-LI0-52,2022-09-26 06:00:00,2022-10-07 13:00:00,PT60M,1,950
4,Cancelled,TQkAi1gcCs2HwFhTeB9a_w,2,Planned maintenance,NL,BE,MAW,22T-BE-IN-LI0-52,2022-10-07 13:00:00,2022-10-07 13:30:00,PT15M,1,950
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,Cancelled,CJoRU3JiGV2_35xzE3y0yw,2,Planned maintenance,HU,AT,MAW,14T-380-0-0440AX,2022-11-16 05:00:00,2022-11-16 19:00:00,PT60M,1,800
1225,Cancelled,i-vLwk_IPCwlV56J-pyk6Q,2,Planned maintenance,HU,AT,MAW,10T-AT-CZ-000010,2022-10-10 04:00:00,2022-10-14 18:00:00,PT60M,1,900
1226,Cancelled,wZltkw45YiTSfEP28mksvA,2,Planned maintenance,HU,AT,MAW,14T-380-0-0439AA,2022-11-15 05:00:00,2022-11-15 19:00:00,PT60M,1,800
1227,Cancelled,sHZ6-NkBQVu5wbA-RZ4kRw,2,Planned maintenance,HU,AT,MAW,10T-AT-HU-00002U,2022-10-03 04:00:00,2022-10-28 18:00:00,PT60M,1,900


In [252]:
df_transmission_outages[(df_transmission_outages.docstatus != 'Cancelled')].to_excel('./transmission_outages.xlsx')

In [266]:
df_ptdf_k_closest = df_ptdf.copy()

for index, row in df_ptdf_k_closest.iterrows():
    closest_nodes = df_grid.loc[row['line_id'], "closest_nodes_" + str(k_closest)]
    closest_lines = df_grid.loc[row['line_id'], "closest_edges_" + str(k_closest)]
    
    for p in range(1, k_closest+1):
        total_available_capacity = 0
        plants = df_plants[df_plants.node == closest_nodes[p-1]]
        for p_i, plant in plants.iterrows():
            available_capacity = plant['installed_capacity']
            df_plant_outages = df_production_outages[(df_production_outages.docstatus != 'Cancelled') & (df_production_outages.production_resource_id == plant['eic'])]
            if len(df_plant_outages) > 0:
                df_plant_outage = df_plant_outages[(df_plant_outages.start <= row['DateTime']) & (df_plant_outages.end >= row['DateTime'])]
                if len(df_plant_outage) > 0:
                    available_capacity = df_plant_outage.iloc[0]['avail_qty']

            total_available_capacity += float(available_capacity)
                    
        df_ptdf_k_closest.loc[index, "p" + str(p)] = total_available_capacity

    for t in range(1, k_closest):
        has_outage = 0
        if len(closest_lines) >= t:
            eic = df_grid.loc[closest_lines[t-1], 'eic']
            df_line_outages = df_transmission_outages[(df_transmission_outages.docstatus != 'Cancelled') & (df_transmission_outages.eic == eic)]
            if len(df_line_outages) > 0:
                df_line_outage = df_line_outages[(df_line_outages.start <= row['DateTime']) & (df_line_outages.end >= row['DateTime'])]
                if len(df_line_outage) > 0:
                    has_outage = 1
                
        df_ptdf_k_closest.loc[index, "t" + str(t)] = has_outage

df_ptdf_k_closest = df_ptdf_k_closest[["p" + str(p) for p in range(1, k_closest+1)] + ["t" + str(t) for t in range(1, k_closest)]]

display(df_ptdf_k_closest)

df_ptdf_k_closest[["t" + str(t) for t in range(1, k_closest)]].sum()

KeyboardInterrupt: 

In [133]:
for i, (p_index, plant) in enumerate(df_plants.iterrows()):
    print(i)
    available_capacity = plant['installed_capacity']
    df_plant_outages = df_outages[(df_outages.docstatus != 'Cancelled') & (df_outages.production_resource_id == plant['eic'])]
    has_outage = len(df_plant_outages) > 0
    
    for j, (t_index, time_row) in enumerate(df_gen.iterrows()):
        if has_outage:
            df_plant_outage = df_plant_outages[(df_plant_outages.start <= t_index) & (df_plant_outages.end >= t_index)]
            if len(df_plant_outage) > 0:
                available_capacity = df_plant_outage.iloc[0]['avail_qty']

        availability_matrix[j][i] = available_capacity

,biomass,brown_coal,coal_gas,natural_gas,hard_coal,oil,hydro,nuclear,waste,other,solar,wind_onshore,wind_offshore,other_renewable
2022-09-01 00:00:00,0,0,0,1405,0,0,662.5,0,0,0,0,0,0,0
2022-09-01 01:00:00,0,0,0,1405,0,0,182.5,0,0,0,0,0,0,0
2022-09-01 02:00:00,0,0,0,1405,0,0,182.5,0,0,0,0,0,0,0
2022-09-01 03:00:00,0,0,0,1405,0,0,182.5,0,0,0,0,0,0,0
2022-09-01 04:00:00,0,0,0,1405,0,0,182.5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-30 19:00:00,0,0,0,305,0,0,200.2,0,0,0,0,0,0,0
2023-03-30 20:00:00,0,0,0,305,0,0,200.2,0,0,0,0,0,0,0
2023-03-30 21:00:00,0,0,0,305,0,0,200.2,0,0,0,0,0,0,0
2023-03-30 22:00:00,0,0,0,305,0,0,200.2,0,0,0,0,0,0,0
